#**[Predict Air Quality with Machine Learning](https://www.sciencebuddies.org/science-fair-projects/project-ideas/)**

This notebook was developed by Science Buddies [www.sciencebuddies.org](https://www.sciencebuddies.org/) as part of a science project to allow students to explore and learn about artificial intelligence. For personal use, this notebook can be downloaded and modified with attribution. For all other uses, please see our [Terms and Conditions of Fair Use](https://www.sciencebuddies.org/about/terms-and-conditions-of-fair-use).  

**Troubleshooting tips**
*   Read the written instructions at Science Buddies and the text and comments on this page carefully.
*   If you make changes that break the code, you can download a fresh copy of this notebook and start over.

*   If you are using this notebook for a science project and need help, visit our [Ask an Expert](https://www.sciencebuddies.org/science-fair-projects/ask-an-expert-intro) forum for assistance.

## **How To Use This Notebook**

This notebook contains text fields, like this one, that give you information about the project and instructions.

In [ ]:
# There are also code blocks, like this one.

# The green text in a code block are comments. Comments are descriptions of what the code does.

# The non-green text in a code block is the Python code. Click on the triangle in the top left corner to run this code block.

print("Congratulations, you ran a code block! Try changing the text in the code and running it again.")

# Importing Libraries

In [ ]:
# Standard Library
import os

# Data Science Libraries
import pandas as pd
import numpy as np

# Plotting Libraries
import matplotlib.pyplot as plt

# Machine Learning Libraries (Scikit-learn)
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Deep Learning Libraries (TensorFlow/Keras)
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import InputLayer, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 3. Loading the Data into a Pandas DataFrame

Code Block 3A

In [ ]:
# Define the path to the CSV file containing air quality data
csv_path = '/content/drive/MyDrive/Air Quality Prediction/aqidaily_fiveyears.csv'

# Read the CSV file into a DataFrame using pandas
df = pd.read_csv(csv_path)

# Display the first 5 rows of the DataFrame to check the data
df.head()

# 4. Preprocessing

Code Block 4A

In [ ]:
# Convert the 'Date' column to datetime format and set it as the DataFrame index
df.index = pd.to_datetime(df['Date'])

# Display the first 5 rows of the updated DataFrame to verify the changes
df.head()

Code Block 4B

In [ ]:
# Extract the 'Overall AQI Value' column from the DataFrame and store it in a variable 'temp'
temp = df['Overall AQI Value']

# Plot the values in the 'temp' series to visualize the trend of the AQI over time
temp.plot()

Code Block 4C

In [ ]:
# Drop the unnecessary columns
df.drop(columns=['Site Name (of Overall AQI)'], inplace=True)

df.head()

Code Block 4D

In [ ]:
# Convert columns to numeric, coercing errors to NaN
df['CO'] = pd.to_numeric(df['CO'], errors='coerce')
df['Ozone'] = pd.to_numeric(df['Ozone'], errors='coerce')
df['PM10'] = pd.to_numeric(df['PM10'], errors='coerce')
df['PM25'] = pd.to_numeric(df['PM25'], errors='coerce')
df['NO2'] = pd.to_numeric(df['NO2'], errors='coerce')

# Replace '-' with NaN
df.replace('-', np.nan, inplace=True)

# Fill Null values (if any) with last known number
df.ffill(inplace=True)

df.head()

Code Block 4E

In [ ]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# TODO: Select the columns to normalize
columns_to_normalize = ['CO', 'Ozone', 'PM10', 'PM25', 'NO2']

# Fit and transform the data
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

df.head()

Code Block 4F

In [ ]:
def df_to_X_y(df, window_size=5):
    # Convert the input dataframe to a NumPy array for easier manipulation
    df_as_np = df.to_numpy()

    # Initialize empty lists to hold the features (X), labels (y), and indices
    X = []
    y = []
    indices = []

    # Iterate over the length of the dataframe minus the window size
    for i in range(len(df_as_np) - window_size):
        # Extract a 'window' of values from the dataframe (a slice of rows)
        # and format each value as a list (to keep consistent dimensionality)
        row = [[a] for a in df_as_np[i:i + window_size]]

        # Append the windowed rows to the feature list X
        X.append(row)

        # The label (y) is the value that follows the current window
        label = df_as_np[i + window_size]

        # Append the label to the label list y
        y.append(label)

        # Append the starting index of the window to the indices list
        indices.append(df.index[i])

    # Convert the feature (X) and label (y) lists into NumPy arrays and return them along with the indices
    return np.array(X), np.array(y), np.array(indices)

Code Block 4G

In [ ]:
# TODO: Change the window size to reflect how many days in advance we want to predict AQI
WINDOW_SIZE = 365  # Define the size of the sliding window (number of time steps to consider for each sequence)

# X will contain sequences of length 'WINDOW_SIZE', and y will contain the corresponding labels (next values)
X, y, indices = df_to_X_y(temp, WINDOW_SIZE)

# Output the shapes of X and y to check the dimensions
X.shape, y.shape

## 5. Split to Train, Validation, and Test

Code Block 5A

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming X, y, and indices are already defined
indices = np.arange(len(X))  # Create an array of indices (or use the actual indices if available)

# Step 1: Split the dataset into training+validation and test sets, preserving indices
X_train_val, X_test, y_train_val, y_test, indices_train_val, indices_test = train_test_split(
    X, y, indices, test_size=0.2, random_state=42
)

# Step 2: Split the training+validation set into training and validation sets, preserving indices
X_train, X_val, y_train, y_val, indices_train, indices_val = train_test_split(
    X_train_val, y_train_val, indices_train_val, test_size=0.25, random_state=42
)

# Now you have the features (X), labels (y), and corresponding indices for each split.

Code Block 5B

In [ ]:
# Now, you have:
# X_train, y_train: Training set (60%)
# X_val, y_val: Validation set (20%)
# X_test, y_test: Test set (20%)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# 6. Training the Model

Code Block 6A

In [ ]:
# Create a Sequential model (builds the model layer by layer)
model = Sequential()

# Add an input layer that expects sequences of length WINDOW_SIZE with 1 feature per time step
model.add(InputLayer(input_shape=(WINDOW_SIZE, 1)))

# Add an LSTM layer with 64 units (neurons) to learn patterns from the sequential data
model.add(LSTM(64))

# Add a Dense (fully connected) layer with 8 neurons and ReLU activation to introduce non-linearity
model.add(Dense(8, activation='relu'))

# Add a final Dense layer with 1 neuron and linear activation to produce the predicted output (one value)
model.add(Dense(1, activation='linear'))

# Print a summary of the model's layers and the number of parameters in each layer
model.summary()

Code Block 6B

In [ ]:
# Create a ModelCheckpoint callback to save the model to 'model.keras' only when its performance improves
cp = ModelCheckpoint('/content/drive/MyDrive/Air Quality Prediction/model.keras', save_best_only=True)

# Compile the model, specifying the loss function, optimizer, and evaluation metric
# - loss=MeanSquaredError(): Measures how well the model is performing by calculating the average squared difference between predictions and actual values
# - optimizer=Adam(learning_rate=0.001): Adam optimizer adjusts the learning rate during training; here we set the initial learning rate to 0.001
# - metrics=[RootMeanSquaredError()]: The model will evaluate its performance using RMSE, which is a common metric in regression tasks
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

Code Block 6C

In [ ]:
# Train the model using the training data (X_train, y_train)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[cp])

# 7. Evaluating the Model

Code Block 7A

In [ ]:
# Load the pre-trained model from the specified file path
model = tf.keras.models.load_model('/content/drive/MyDrive/Air Quality Prediction/model.keras')

Code Block 7B

In [ ]:
y_pred = model.predict(X_test).flatten()

# Set the starting date
# start_date = pd.to_datetime('2019-01-01')
start_date = pd.to_datetime(df.index.min())

# Create a date range that covers the maximum index
max_index = indices_test.max()
date_range = pd.date_range(start=start_date, periods=max_index + 1)

# Use the indices to select corresponding dates from the date range
date_index = date_range[indices_test]

# Example data
data = {'Date': date_index, 'y_test': y_test, 'y_pred': y_pred}
df = pd.DataFrame(data)

# Set 'Date' as the index
df.set_index('Date', inplace=True)

# Sort the DataFrame by the index (Date) in ascending order
df.sort_index(ascending=True, inplace=True)

# View the sorted DataFrame
df

Code Block 7C

In [ ]:
# Plot the values against the date index
plt.figure(figsize=(10, 6))

# Plot y_test
plt.plot(df.index, df['y_test'], label='Actual Values (y_test)', color='blue', marker='o')

# Plot y_pred
plt.plot(df.index, df['y_pred'], label='Predicted Values (y_pred)', color='red', linestyle='--', marker='x')

# Customize the plot
plt.title('y_test vs. y_pred over Time')
plt.xlabel('Date')
plt.ylabel('Values')
plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()

Code Block 7D

In [ ]:
# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

Code Block 7E

In [ ]:
# Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

Code Block 7F

In [ ]:
# Calculate R-squared
r2 = r2_score(y_test, y_pred)

# Fit a best-fit line
model = LinearRegression()
y_test_reshaped = np.array(y_test).reshape(-1, 1)  # Reshape for fitting
model.fit(y_test_reshaped, y_pred)
y_best_fit = model.predict(y_test_reshaped)

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color='blue', alpha=0.5, label="Predicted vs. Actual")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--', label="Perfect Prediction")
plt.plot(y_test, y_best_fit, color='green', linestyle='-', label="Best Fit Line")
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.title(f"True vs Predicted Values (R² = {r2:.2f})")
plt.legend()
plt.show()